# Exercises

## Thermodynamic states and simple processes

Calculate ...

1. ... density and enthalpy of R290 at 5 bars pressure and 50 °C.
2. ... the enthalpy of saturated liquid Ammonia at 75 °C.
3. ... the vapor mass fraction in the two-phase region of Ammonia at the same
   enthalpy as in task 2 but at a temperature of 25 °C.
4. At what temperature does Ammonia start to boil under a pressure of 4 bars.
5. What pressures correspond to a saturation temperature of 20 °C, 75 °C and
   110 °C for water?


In [ ]:
from CoolProp.CoolProp import PropsSI

In [ ]:
PropsSI("D", "P", 5e5, "T", 50 + 273.15, "R290")

In [ ]:
h = PropsSI("H", "Q", 0, "T", 75 + 273.15, "ammonia")
h

In [ ]:
p = PropsSI("P", "T", 25 + 273.15, "Q", 0, "ammonia")
PropsSI("Q", "H", h, "P", p, "ammonia")

In [ ]:
PropsSI("T", "P", 4e5, "Q", 0, "ammonia") - 273.15

In [ ]:
PropsSI("P", "T", [t + 273.15 for t in [20, 75, 110]], "Q", 0, "water")


## Refrigerants

1. Plot the saturation curve for Ammonia, Water, R134a and Pentane in a log-p,T
   diagram for temperature ranging from -25 °C to 100 °C.
2. Compare the plots: What factors may restrict the usage of the working fluids
   in heat pumps?
3. Calculate the entropy for the dew line and the bubble line of the same
   working fluids for temperature values larger than -25 °C. Plot the lines in
   a T-s diagram.
4. Compare the plots: What factors may restrict the usage of the working fluids
   in heat pumps?
   

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
fluids = ["ammonia", "water", "R134a", "Pentane", "R290"]
temperature = np.linspace(-25, 125)
saturation_pressure = {}
for fluid in fluids:
    saturation_pressure[fluid] = PropsSI("P", "T", temperature + 273.15, "Q", 0, fluid)

fig, ax = plt.subplots(2, figsize=(12, 8))

for fluid in fluids:
    ax[0].plot(temperature, saturation_pressure[fluid], label=fluid)
    ax[1].plot(temperature, saturation_pressure[fluid] / saturation_pressure[fluid].min())

ax[0].set_yscale("log")
ax[0].set_ylabel("pressure in Pa")
ax[1].set_xlabel("temperature in °C")

ax[0].plot(temperature, [1e5] * len(temperature), "--", label="ambient pressure")
ax[0].legend()

_ = ax[1].set_ylabel("pressure ratio p / p$_\\text{min}$")

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))

for fluid in fluids:
    temperature_range = np.linspace(-25 + 273.15, PropsSI("TCRIT", fluid))
    bubble_entropy = PropsSI("S", "T", temperature_range, "Q", 0, fluid)
    dew_entropy = PropsSI("S", "T", temperature_range, "Q", 1, fluid)
    _ = ax.plot(bubble_entropy, temperature_range - 273.15, label=fluid)
    color = _[0].get_color()
    ax.plot(dew_entropy, temperature_range - 273.15, color=color)

ax.set_ylabel("temperature in °C")
ax.set_xlabel("entropy in J/kgK")
_ = ax.legend()

## Component Models

### Compressor

Implement a model, that allows you to model a compressor, where the compression
process is adiabatic and reversible (isentropic).
   
1. What power does the compressor draw if 5 kg/s of air are compressed from
   ambient state (1 bar, 25 °C) to 5 bars.
2. What is the outlet temperature of the air.
3. How much mass can be compressed by the same machine, if 1 MW of power are
   available?

Now consider thermodynamic inefficiencies by incorporating isentropic
efficiency in the model.

4. How does the compressor power requirement change to compress the same air
   mass flow as in the first assignment. Make a plot indicating the change of
   power over the change of the isentropic efficiency.


In [ ]:
fluid = "air"
m = 5
p_1 = 1e5
T_1 = 25 + 273.15

h_1 = PropsSI("H", "P", p_1, "T", T_1, fluid)
s_1 = PropsSI("S", "P", p_1, "T", T_1, fluid)

p_2 = 5e5
h_2s = PropsSI("H", "S", s_1, "P", p_2, fluid)

power = m * (h_2s - h_1)
power

In [ ]:
T_2s = PropsSI("T", "H", h_2s, "P", p_2, fluid)
T_2s

In [ ]:
power = 1e6
m = power / (h_2s - h_1)
m

In [ ]:
eta_s = 0.8
m = 5
h_2 = h_1 + (h_2s - h_1) / eta_s
power = m * (h_2 - h_1)
power

In [ ]:
import numpy as np

eta_s_range = np.linspace(0.6, 0.9)
h_2_range = h_1 + (h_2s - h_1) / eta_s_range
T_2_range = PropsSI("T", "P", p_2, "H", h_2_range, fluid)
power_range = m * (h_2_range - h_1)

fig, ax = plt.subplots(2, figsize=(8, 4.5))

eta_s_range *= 100
power_range /= 1e6
T_2_range -= 273.15

ax[0].plot(eta_s_range, power_range)
ax[0].set_ylabel("power input in MW")
ax[1].plot(eta_s_range, T_2_range)
ax[1].set_ylabel("outlet temperature in °C")
ax[1].set_xlabel("isentropic efficiency in %")

ax[0].grid()
ax[1].grid()


### Heat exchanger

...
